<a href="https://colab.research.google.com/github/Ori4est/auto_labeling_v1.0/blob/main/auto_labelling_demo_1v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import sys
sys.path.append('/content/gdrive/MyDrive/Colab Notebooks/AutoLabeling')
sys.path.append('/content/grive/Shareddrives/TheSEA_tech/AutoLabel')

In [3]:
DEMO_PATH = '/content/gdrive/MyDrive/Colab Notebooks/AutoLabeling'
DATA_PATH = '/content/grive/Shareddrives/TheSEA_tech/AutoLabel/dataset'

%cd $DEMO_PATH
try:
  !pip install -r requirements.txt # version conflict
  !pip install psd_tools
  !pip install PhotoshopAPI
  !pip install tqdm
except:
  print("Something went wrong. version conflit?")

/content/gdrive/MyDrive/Colab Notebooks/AutoLabeling
  Using cached aesara-2.9.3-py3-none-any.whl.metadata (2.6 kB)
  Using cached argcomplete-3.4.0-py3-none-any.whl.metadata (16 kB)
  Using cached array_api_strict-2.0.1-py3-none-any.whl.metadata (1.8 kB)
  Using cached astroid-2.14.2-py3-none-any.whl.metadata (4.7 kB)
  Using cached astropy-5.1-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (8.0 kB)
  Using cached asyncssh-unofficial-0.9.2.tar.gz (80 kB)
  Preparing metadata (setup.py) ... done
  Using cached av-12.3.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.6 kB)
  Using cached black-24.4.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (77 kB)
  Using cached brotlicffi-1.1.0.0-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.7 kB)
  Using cached brotlipy-0.7.0-cp35-abi3-manylinux2010_x86_64.whl.metadata (2.9 kB)
  Using cached cchardet-2.1.7.tar.gz (653 kB)
  Preparing metadata (setup.py) ..

In [ ]:
!pip install PhotoshopAPI==0.3.0
!pip install psd_tools

In [4]:
import os
from tqdm.notebook import tqdm
import json
import cv2
import numpy as np
from itertools import groupby, product
import csv
import argparse
from utils import *
from auto_labelling_multi_prods_v3 import layer_blend

In [9]:
DATA_PATH = '/content/gdrive/Shareddrives/TheSEA_tech/AutoLabel/dataset'
!ls $DATA_PATH
test_data_file = os.path.join(DATA_PATH, "test_1v3.json")
work_dir = os.path.join(os.path.dirname(DATA_PATH), "output")

test_data = json.load(open(test_data_file))["test_batch_1"]
label_dir = DATA_PATH

prod_specs_file = open(os.path.join(label_dir, 'autolabel-algo-data-testing', 'prod_static_spec_test.csv'), newline='', encoding='utf_8') # ISO-8859-1 # latin1
prod_specs = list(csv.DictReader(prod_specs_file))

prod_specs_keys = [item['label_id'] for item in prod_specs]
new_prod_specs = dict(zip(prod_specs_keys, prod_specs))

autolabel-algo-data-testing  test_1v3.json  test-cases-rep


In [16]:
# Paired product image and product hed image
for row in tqdm(test_data):
    prod_img = cv2.imread(os.path.join(DATA_PATH, row['dir_name'][1:], row['prod_img']), -1)
    prod_hed_img = cv2.imread(os.path.join(DATA_PATH, row['dir_name'][1:], row['prod_hed']), -1)
    prod_hed_ref_img = cv2.imread(os.path.join(DATA_PATH, row['dir_name'][1:], row['ref_hed']), -1)
    label_img = cv2.imread(os.path.join(label_dir, row['label_img'][1:]), -1)
    label_id = row['dir_name'].split('/')[-1]
    prod_spec = new_prod_specs[label_id]
    print("label_id", label_id, prod_spec)

    layer_blend(hed_image=prod_hed_img,   # (hed w bg)
                product_hed_image=prod_hed_ref_img.copy(),   # (hed wo bg)
                label_image=label_img,      # (brand logo rgba)
                image=prod_img,       # (prod img)
                prod_name=row['prod_img'].split('.')[0],  # (given)
                product_width=float(prod_spec['product_width']),  # (given) set layer width
                product_height=float(prod_spec['product_length']), # (given) set layer height
                label_width=float(prod_spec['label_width']), # (given)
                label_height=float(prod_spec['label_length']), # (given)
                left_margin=float(prod_spec['left_margin_width']), # (given)
                top_margin=float(prod_spec['top_margin_width']),  # (given)
                save_path=os.path.join(work_dir, label_id),
                save_psd=True)

  0%|          | 0/113 [00:00<?, ?it/s]

label_id HR001 {'brand_name': 'HELENA RUBINSTEIN', 'product_name': 'Replasty-Age-Recovery', 'product_width': '1017', 'product_length': '1042', 'label_width': '306', 'label_length': '378', 'left_margin_width': '360', 'top_margin_width': '408', 'label_id': 'HR001', 'label_name': 'Helena-rubinstein-replasty-age-recovery-label'}
comparing p1, cntr, angle (458.0508769931162, 1655.3946557250727) (482, 553) 1.5925175463912122 -1
1st round find closed contours 5
2nd round find open contours 5
1.5384642625682066e-05 inf
1.4022825606141431e-05 1.5384642625682066e-05
0.03641965366244726 1.4022825606141431e-05
0.0454171314185407 1.4022825606141431e-05
0.04618970861326721 1.4022825606141431e-05
max contour 148352.5 (500, 486) (500, 486)
0 503 647 145069.0
1 503 647 148352.5
The grouped elements : [[(1, 503, 647), (0, 503, 647)]]
1 [[(1, 503, 647), (0, 503, 647)]]
contour shape (500, 486)
contour shape (485, 500)
best contour matching score 0 1.4022825606141431e-05 (1, 503, 647) 148352.5
primer cont

RuntimeError: filesystem error: cannot create directories: Read-only file system [/content/gdrive/Shareddrives/TheSEA_tech/AutoLabel/output]